In [ ]:
#driver.get("https://www.1mg.com/drugs-all-medicines")

In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


Threading and multiprocessing

In [ ]:
import threading as th
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
import csv 
import os
import multiprocessing as m
from multiprocessing import Process, Value
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#Enter range like a-b
rng ='s-s'

split = rng.split('-')
s=[]
for i in split:
  s.append(int(ord(i)))
  
if(len(s)==2 and s[0]<=s[1] and s[0]>=97 and s[1]>=97 and s[0]<=122 and s[1]<=122):
  print("Right input.")
else:
  print("Please try again.")


Right input.


In [ ]:
#all global arrays
arr_length = s[1]-s[0]+1
where_to_start=[0]*arr_length
driv = []
driv1 = []
def make_file(i):
  driv.append(webdriver.Chrome('chromedriver',options=chrome_options))
  driv1.append(webdriver.Chrome('chromedriver',options=chrome_options))
  if(os.path.isfile("/content/gdrive/MyDrive/1mg_drug_data/output_"+chr(97+i)+".csv")==False):
    print("Creating output_"+chr(97+i)+".csv")
    where_to_start.append(0)
    with open("/content/gdrive/MyDrive/1mg_drug_data/output_"+chr(97+i)+".csv", mode='w') as f:
      file = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
      file.writerow(["Name", "Price", "Prescription", "Quantity", "Company", "Chemicals", "Availability", "Temprature", "Usage", "Side effect","link","Alcohol","Pregnancy","Breast feeding","Driving","Kidney","Liver","Alcohol dec.","Pregnancy dec.","Breast feeding dec.","Driving dec.","Kidney dec.","Liver dec."])
  else:
    df = pd.read_csv("/content/gdrive/MyDrive/1mg_drug_data/output_"+chr(97+i)+".csv")
    df = df.iloc[:-1]
    df.to_csv("/content/gdrive/MyDrive/1mg_drug_data/output_"+chr(97+i)+".csv", index=False)
    print("File output_"+chr(97+i)+".csv already exists and has "+str(len(df.index))+" drug data.")
    where_to_start[i-s[0]+97]=len(df.index)

threads=[]
for i in range(s[0]-97,s[1]-96):
  t = th.Thread(target=make_file,args=(i,))
  threads.append(t)
  t.start()

for tr in threads:
  tr.join()
print("Finished")


File output_s.csv already exists and has 13020 drug data.
Finished


In [ ]:
def start(index,alpha):
  cnt=0
  warning=[""]*6
  des=[""]*6
  pro = "No data"
  storage = "(//div[@class='saltInfo DrugHeader__meta-value___vqYM0'])[2]"
  usage = "//ul[@class='DrugOverview__list___1HjxR DrugOverview__uses___1jmC3']"
  sideeffect = "//div[@class='DrugOverview__list-container___2eAr6 DrugOverview__content___22ZBX']/ul"
  
  s = (where_to_start[index]%30) + 1
  where_to_start[index]=0
  for i in range (s,31):
    box = "(//div[@class='style__product-card___1gbex style__card___3eL67 style__raised___3MFEA style__white-bg___10nDR style__overflow-hidden___2maTX']/a)["+str(i)+"]"
    try:
      data = driv[index].find_element_by_xpath(box).text
    except Exception as e:
      data = "No data"

    try:
      link = driv[index].find_element_by_xpath(box).get_attribute("href");
    except Exception as e:
      link = "No data"

    try:
      driv1[index].get(link)
    except Exception as e:
      link = "No data"

    cnt = 0
    while(cnt<6):
      cnt=cnt+1
      link2 = "(//div[@class='DrugOverview__content___22ZBX']/div[@class='DrugOverview__warning-top___UD3xX']/div[2])["+str(cnt)+"]"
      link3 = "(//div[@class='DrugOverview__content___22ZBX']/div[@class='DrugOverview__content___22ZBX'])["+str(cnt)+"]"
      try:
        warning[cnt-1] = driv1[index].find_element_by_xpath(link2).text
      except Exception as e:
        warning[cnt-1] = "No data"
      try:
        des[cnt-1] = driv1[index].find_element_by_xpath(link3).text
      except Exception as e:
        des[cnt-1] = "No data"
            
    try:
      data1 = driv1[index].find_element_by_xpath(storage).text
    except Exception as e:
      data1 = "No data"

    try:
      data2 = driv1[index].find_element_by_xpath(usage).text 
    except Exception as e:
      data2 = "No data"

    try:
      data3 = driv1[index].find_element_by_xpath(sideeffect).text 
    except Exception as e:
      data3 = "No data"

    split = data.split("\n")
    with open("/content/gdrive/MyDrive/1mg_drug_data/output_"+chr(97+alpha)+".csv", mode='a') as f:
      file = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
      if(len(split)>6):
        file.writerow([split[0], split[1], split[2], split[3], split[4], split[5], split[6], data1, data2, data3,link,warning[0],warning[1],warning[2],warning[3],warning[4],warning[5],des[0],des[1],des[2],des[3],des[4],des[5]])
      else:
        file.writerow([split[0], split[1], "", split[2], split[3], split[4], split[5], data1, data2, data3,link,warning[0],warning[1],warning[2],warning[3],warning[4],warning[5],des[0],des[1],des[2],des[3],des[4],des[5]])
      print("Drug "+split[0]+" added in output_"+chr(97+alpha)+".csv")


def threadfunction(index,alpha):
  pageno = int(where_to_start[index]/30)
  while(True):
    try:
      pageno = pageno+1
      link2 = "https://www.1mg.com/drugs-all-medicines?page="+str(pageno)+"&label="+chr(97+alpha)
      print(link2)
      driv[index].get(link2)
      start(index,alpha)  
    except Exception as e:
      print("Error")
      break


In [ ]:
def th1():
  threadlist = []
  for i in range(0,arr_length):
    t = th.Thread(target=threadfunction,args=(i,s[0]-97+i,))
    threadlist.append(t)
    t.start()

  for tr in threadlist:
    tr.join()

th1()
print("Finished.Check the drive for data.")

https://www.1mg.com/drugs-all-medicines?page=435&label=s
Drug Sibatax 260mg Injection added in output_s.csv
Drug Sidzox 4 Capsule added in output_s.csv
Drug Steicef DS 100mg Dry Syrup added in output_s.csv
Drug StayHappi Hydrochlorothiazide+Olmesartan Medoxomil 12.5mg/20mg Tablet added in output_s.csv
Drug Samzone 500mg Injection added in output_s.csv
Drug Serchcef-O Injection added in output_s.csv
Drug Serchcef XP Injection added in output_s.csv
Drug Sbdol TH 100mg/4mg Tablet added in output_s.csv
Drug Sancef 250mg Injection added in output_s.csv
Drug Seadrox 250mg Tablet added in output_s.csv
Drug Sertimo 50mg Tablet added in output_s.csv
Drug S T Dase Plus 50mg/10mg Tablet added in output_s.csv
Drug Siffy Clav 200mg/125mg Tablet added in output_s.csv
Drug Shubhcyp Syrup added in output_s.csv
Drug Sonoxone 250mg Injection added in output_s.csv
Drug Symyx 500mg Injection added in output_s.csv
Drug Sanpip T 4000mg/500mg Injection added in output_s.csv
Drug Saramox CV 250mg/125mg Tablet